# Data Cleaning 

In [ ]:
import pandas as pd

# Read the users dataset.

Take a look at what is the `users.csv` separator.

In [ ]:
users = pd.read_csv('users.csv', delimiter='#')

## Check its shape

See the number of rows and columns you're dealing.

In [ ]:
users.shape

## Use the .head() to see some rows of your dataframe.

In [ ]:
users.head(5)

## Get the data info. 

Which columns have a great number of missing values? How many space does this dataframe is occupying in your memory?

In [ ]:
users.info()

## Rename Id column to user_id.

Remember to store you results back at the dataframe.

In [ ]:
users.rename(columns={'Id': 'user_id'}, inplace=True)

# Import the `posts.csv` dataset.

Note that this is a `gzip compressed csv`. In order to read this file correctly, you'll have to read the documentation (or help) of your `pd.read_csv()` function and check the `compression` argument. Try to understand which value of `compression=...` you should put in order to read your dataframe. 

In [ ]:
posts = pd.read_csv('posts.csv.gzip', compression='gzip')

## Perform the same as above to understand a bit of your data (head, info, shape)

In [ ]:
posts.shape

In [ ]:
posts.head(5)

In [ ]:
posts.info()

## Rename Id column to post_id and OwnerUserId to user_id.

Again, remember to check that your results are correctly stored inside the dataframe.

In [ ]:
posts.rename(columns={'Id': 'post_id', 'OwnerUserId': 'user_id'}, inplace=True)

## Define new dataframes for users and posts with the following selected columns:

**users columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts columns**: postId, Score, userID, ViewCount, CommentCount, Body

In [ ]:
new_users = users[['user_id', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
new_posts = posts[['post_id', 'Score', 'user_id', 'ViewCount', 'CommentCount', 'Body']]

**Note:** Check the new posts dataframe's info. What is the most noticeable change? 

Explain why we have chosen only some columns of it in terms of efficiency.

In [ ]:
posts.info()

In [ ]:
new_posts.info()
# Less memory

# Merge the new dataframes you have created, of users and posts. Create a dataframe called `posts_from_users`

You will need to make an inner [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes. 

Think carefully which should be the key(s) for your merging.

In [ ]:
posts_from_users = new_posts.merge(new_users)

## Check the number of duplicated rows.

Remember you can sum the results of a mask to get how many numbers the True value appeared in the results. This occurs because `True` is interpreted as `1` in Python whereas `False` is interpreted as `0`.

In [ ]:
posts_from_users.duplicated().sum()

## Find those duplicate values and try to understand what happened.

*Hint:* You can use the argument `keep=False` from the `.duplicated()` method to bring the duplication.

*Hint 2:* You can sort the values `by=['user_id', 'post_id']` to see them in order.


In [ ]:
posts_from_users[
    posts_from_users.duplicated(keep=False)
] \
    .sort_values(by=['user_id', 'post_id'])

In [ ]:
# new_users seems to have duplicate records
new_posts.duplicated().sum(), new_users.duplicated().sum()

In [ ]:
# new_users HAS duplicate records
new_users[
    new_users.duplicated(keep=False)
] \
    .sort_values(by='user_id')

## Should you drop it? If you think it is reasonable to drop it, then drop it.

Think: How would you correct it in the first place? That is, what was wrong in the first place?

*Hint:* There's a pandas method to drop duplicates. If you wanted to do it by hand, you could select the indexes of the duplicated values and `.drop()` it. 

In [ ]:
# rebuild post_from_users dropping duplicates from new_users
posts_from_users = new_posts.merge(
    new_users.drop_duplicates()
)

In [ ]:
# did it work?
posts_from_users.duplicated().sum()

## 10. How many missing values do you have in your merged dataframe? On which columns?

In [ ]:
posts_from_users.isnull().sum()

## Select only the rows in which there at least some missing values.

In [ ]:
posts_from_users[
    posts_from_users \
        .isnull() \
        .any(axis = 1)
]

## You will need to make something with missing values.  Will you clean or filling them? 

Pay attention. There can be different reasons for the missings numbers. Look at the `user_id` of some of them, look at the body of the message. Which ones you're sure of what should be and which one can you infer? Don't hurry up, take a look at your data.

In [ ]:
# Will set ViewCount to 0
posts_from_users = \
    posts_from_users.fillna(value={'ViewCount': 0})

In [ ]:
# Will drop rows with empty Body
posts_from_users = \
    posts_from_users.dropna(subset=['Body'])

## Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [ ]:
posts_from_users.info()
# DUNNO!!!!

# Bonus 1: (filtering) What is the average number of comments for users who are above the average reputation?

*Hint:* Calculate the average of the user Reputation. Store it in a variable called `avg_reputation` and then use that variable for filtering the dataset and generating the results for each case (for the case in which `Reputation > {avg_reputation}` and etc.

*Hint 2:* You could create a variable based on that condition and use the group by function perform the task above.

In [ ]:
avg_reputation = posts_from_users.Reputation.mean()

In [ ]:
posts_from_users[
    posts_from_users.Reputation > avg_reputation
] \
    .CommentCount \
    .mean()

# Bonus 2: (grouping) Group your dataframe by the Reputation of your user. Calculate the mean value of ViewCount and CommentCount for each reputation value.

Suppose the missing values on ViewCount are due a systemic error and you wanted to guess what values should have been there in the first place, but the system abended.

Would that be an interesting candidate for inputting the value for the missing `ViewCount` values? If so, input it with these values.

In [ ]:
a = posts_from_users.groupby(by='Reputation')

In [ ]:
a.mean()[['ViewCount', 'CommentCount']]
# TODO: revisit and try-out the insight on possibity of systemic errors for those NaN ViewCount

## refs

Sample database used: https://relational.fit.cvut.cz/dataset/Stats

Stack-overflow database: https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/
